In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from meteostat import Stations, Daily, Hourly, units

In [2]:
def get_station(lat, long, n=1):
  """Get the weather station(s) nearest to a lat-long coordinate"""
  stations = Stations().nearby(lat, long)
  station = stations.fetch(n)
  return(station)

station = get_station(lat=43.0505, long=-78.8766, n=1)

In [3]:
def get_daily_weather(station, start_date, end = datetime.now()):
  df = Daily(station, start=start_date, end=end).convert(units.imperial).fetch().reset_index()
  return(df)


In [5]:
daily_weather = get_daily_weather(station=station, start_date = datetime.now() - timedelta(days=60+28))

daily_stats = (daily_weather
    .pipe(lambda x: x.assign(prcp = x['prcp'].fillna(0)))
    .pipe(lambda x: x.assign(total_prcp_7_days = x['prcp'].rolling(7).sum()))
    .pipe(lambda x: x.assign(total_prcp_14_days = x['prcp'].rolling(14).sum()))
    .pipe(lambda x: x.assign(total_prcp_28_days = x['prcp'].rolling(28).sum()))
    .pipe(lambda x: x.assign(avg_tmax_7_days = x['tmax'].rolling(7).mean()))
    .pipe(lambda x: x.assign(avg_tmin_7_days = x['tmin'].rolling(7).mean()))
    .loc[:, ['time', 'prcp', 'total_prcp_7_days', 'total_prcp_14_days', 'total_prcp_28_days', 'avg_tmax_7_days', 'avg_tmin_7_days']]
    .dropna()
    .rename(columns = {"time": "index"})
    .set_index(['index'])
)

daily_stats   

,prcp,total_prcp_7_days,total_prcp_14_days,total_prcp_28_days,avg_tmax_7_days,avg_tmin_7_days
index,,,,,,
2021-05-14,0.000,2.040000e-01,1.030,2.574,57.871429,36.842857
2021-05-15,0.000,1.690000e-01,1.014,2.550,60.014286,38.128571
2021-05-16,0.000,5.900000e-02,0.825,2.538,62.871429,38.857143
2021-05-17,0.000,5.500000e-02,0.727,2.510,65.642857,40.757143
2021-05-18,0.000,6.938894e-18,0.542,2.337,69.371429,43.328571
...,...,...,...,...,...,...
2021-07-09,0.217,2.220000e+00,3.877,4.945,79.114286,63.557143
2021-07-10,0.059,2.196000e+00,3.408,5.000,79.114286,64.585714
2021-07-11,1.610,3.775000e+00,5.018,6.523,77.314286,64.585714
